In [30]:
import pandas as pd
import numpy as np

In [31]:
import devanalyst.simulation.GenerateTimecards as timecard
import devanalyst.simulation.statics as S_

<h1>Stochastic Utilities</h1>

In [32]:
# Helper class to make random choices, but with a consistent generator instance across all calling sequences 
# so that if required a deterministic output is produced system-wide from a single seed.
class Random():
    def __init__(self):
        self.seed = None
        self.random = np.random.RandomState()
        
    def reset(self, seed):
        self.seed = seed
        self.random = np.random.RandomState(self.seed)
        
    # Returns a random element from an array. Returns None if array is empty
    def pickOne(self, array):
        if len(array)==0:
            return None
        return array[self.random.randint(0, len(array))]

    # Returns a random index from a Pandas Series
    def pickOneIdx(self, series):
        return series.index[self.random.randint(0, len(series))]   

    # Returns an integer corresponding to a random duration between 1 day and the maxDuration. 
    def pickHowLong(self, maxDuration):
        return self.random.randint(1, maxDuration +1)    

In [33]:
class ModelsConfig:
    
    # -costModels: array of CostModel instances, each representing an independent driver for how actual costs deviate from
    # estimates. Thus the real cost is obtained by applying all the models in succession to the estimate.
    # -qualityModels: TBD
    # -allocationModel: an instance of an AllocationModel
    def __init__(self, costModels, qualityModels, allocationModel):
        self.costModels        = costModels
        self.qualityModels     = qualityModels
        self.allocationModel  = allocationModel
        
        self.random = Random()

<h1>Cost Models</h1>

In [34]:
# Abstract class
class CostModel:
    
    # Returns a cost multipier for the WorkItem 'item', i.e., a number equal to the ratio between the 'actual cost'
    # (in man-days) of developing the 'item' and the 'estimated cost'
    #
    # -item: a WorkItem
    def runModel(self, item): 
        return # This is the abstract class, so this method should never be called as concrete class implement it.

In [35]:
class DefaultCostModel (CostModel):
# Default class for models that simulate how actual costs differ from estimates. Usually there might be different
# implementations, each trying to capture a different dynamic with a different driver. This default class just assumes
# everything takes longer than expected by a given delay

    # -delay_pct: the percentage by which estimates are off. For example, a delay_pct of 0.25 means that a task
    # estimated to take 10 man-days actually takes 12.5 man-days.
    def __init__(self, delay_pct = 0.25):
        self.delay_pct = delay_pct
        return  
    # 
    def runModel(self, item):   
        return 1 + self.delay_pct

In [36]:
def computeRealCost(item, costModels):
# Computes the real cost of delivering a work item, based on a number of 'factors', which are functions implementing a model
# for what drives costs to differ from estimates.

    cost = item.estimate;
    for m in costModels:
        cost *= m.runModel(item)
    return cost

<h1>Allocation Models</h1>

In [37]:
# Abstract parent class
class AvailabilityCriterion:
    def getPeopleWithBandwidth(self, timeRequired, sprintDuration):
        return

In [38]:
class GreedyAvailabilityCriterion (AvailabilityCriterion):  
    
    def __init__(self, work, currentOrNext):
        self.work = work
        self.currentOrNext = currentOrNext
    
        return
    
    def getPeopleWithBandwidth(self, timeRequired, sprintDuration):
        if self.currentOrNext:
            available = self.work.committedTime(sprintDuration)[['Developer', 'Bandwidth']]
            return list(available[available['Bandwidth'] >= timeRequired]['Developer'])
            
        else:
            available = self.work.committedTime(sprintDuration)[['Developer', 'NEXT SPRINT Bandwidth']]
            # Filter to only developers who have 'carry over' bandwidth from this spring into the next one
            haveCarryOver = available[available['NEXT SPRINT Bandwidth'] > sprintDuration] 
            return list(haveCarryOver[haveCarryOver['NEXT SPRINT Bandwidth'] >= timeRequired]['Developer'])  

In [39]:
class AllocationModel:
# Abstract class 

    # Implemented by concrete classes.
    # Mutates work allocating WorkItem's to developers
    def allocate(self, work, modelsConfig): 
        return

In [40]:
class GreedyAllocationModelHelper:   
    #
    # -workToPick: an array of WorkItem objects, corresponding to unplanned tasks that are candidate tasks to allocate
    # to developers
    # -work: a WorkAssignment instance reflecting a ScrumTeam's allocations and remaining bandwidth prior to the
    # allocation this model will conduct. This model's allocation process will then mutate 'work' by reflecting in it
    # tasks that are no longer unplanned but rather are now allocated to a particular developer.
    # -availabilityCriterion: instance of AvailabilityCriterion
    # -currentOrNext: boolean to indicate if the allocation is being done for the current sprint or the next sprint. True 
    # for current, False for next.
    def __init__(self, workToPick, currentOrNext, sprintDuration):
        self.workToPick              = workToPick
        self.currentOrNext           = currentOrNext
        self.sprintDuration          = sprintDuration
        
        return
    
    # Mutates 'work' and 'self.workToPick' by allocating WorkItem's to developers and in the process depleting
    # partially or fully the 'self.workToPick'
    def allocate(self, work, modelsConfig): 
        availabilityCriterion   = GreedyAvailabilityCriterion(work, self.currentOrNext)        
        while len(self.workToPick) > 0:
            
            #item = pickOne(self.workToPick)
            item = modelsConfig.random.pickOne(self.workToPick)
            
            self.workToPick.remove(item) # In next cycle of loop don't want to encounter this item, as it would be processed by then
            timeRequired = item.estimate * (1-item.percentAchieved)
        
            peopleWithTimeToDoIt = availabilityCriterion.getPeopleWithBandwidth(timeRequired, self.sprintDuration)
            
            #potentialOwner = pickOne(peopleWithTimeToDoIt)
            potentialOwner = modelsConfig.random.pickOne(peopleWithTimeToDoIt)
            
            if potentialOwner == None:
                # This WorkItem can't be done in this sprint, as nobody has enough time for the effort it requires.
                # Try with some other work item
                 continue
            
            if self.currentOrNext:
                work.reAssign(item, potentialOwner, S_.CURRENT_SPRINT)
            else:
                work.reAssign(item, potentialOwner, S_.NEXT_SPRINT)   

In [41]:
class GreedyAllocationModel (AllocationModel):
#class NEWGreedyAllocationModel (AllocationModel):

    def __init__(self, sprintDuration):
        self.sprintDuration = sprintDuration
        return
        
    def allocate(self, work, modelsConfig):
        unplanned = work.allocations[S_.UNPLANNED][S_.OWNER_TBD]
        workToPick = []
        workToPick.extend(unplanned[S_.BUGS_ON_COMPLETED_STORIES])
        workToPick.extend(unplanned[S_.BUGS_ON_UNFINISHED_STORIES])
        workToPick.extend(unplanned[S_.UNFINISHED_STORIES])   

        helper = GreedyAllocationModelHelper(workToPick, True, self.sprintDuration)
        helper.allocate(work, modelsConfig) # mutates 'work' and 'workToPick'

        # Now try again, but this time allocating any unused time to deliverables for the next sprint, i.e., use
        # time left over from the current sprint to get a heat start on the work for the next sprint, borrowing next sprint's
        # capacity since we only need to deliver then

        #Update unplanned, workToPick since we changed it in prior call to helper
        unplanned = work.allocations[S_.UNPLANNED][S_.OWNER_TBD] 
        workToPick = []
        workToPick.extend(unplanned[S_.BUGS_ON_COMPLETED_STORIES])
        workToPick.extend(unplanned[S_.BUGS_ON_UNFINISHED_STORIES])
        workToPick.extend(unplanned[S_.UNFINISHED_STORIES]) 

        #helper = GreedyAllocationModelHelper(workToPick, False, self.sprintDuration)
        helper.workToPick = workToPick
        helper.currentOrNext = False
        helper.allocate(work, modelsConfig) # mutates 'work' and 'workToPick'    
        
        return work